In [ ]:
import pandas as pd
import datetime
import json
#import pandas_datareader as pdr
import numpy as np
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import optimizers
from matplotlib.image import imread
from keras.preprocessing import image
from sklearn.metrics import r2_score
import seaborn as sns

### Pulling stock list from Virtus LifeSci Biotech ETF

In [ ]:
stocks = ['ITCI', 'AXSM', 'CVM', 'KRTX', 'APLS', 'PRVB', 'CRTX', 'EPZM',
       'CRBP', 'CCXI', 'TGTX', 'IMGN', 'FATE', 'AKBA', 'DNLI', 'RIGL',
       'PRNB', 'ARVN', 'ZYME', 'MRNA', 'CYTK', 'FTSV', 'ASND',
       'XBIT', 'ALEC', 'ORTX', 'ADVM', 'MGNX', 'AKRO', 'ESPR', 'BHVN',
       'YMAB', 'RGNX', 'MYOK', 'TPTX', 'ATNX', 'BLUE', 'AGEN', 'AVRO',
       'DCPH', 'CTMX', 'SRNE', 'ARDX', 'BCRX', 'RETA', 'KOD', 'DTIL',
       'RARX', 'AUTL', 'GTHX', 'CARA', 'KDMN', 'XNCR', 'ACHN', 'ARNA',
       'RCKT', 'TBIO', 'VYGR', 'SGMO', 'ODT', 'ANAB', 'ATRA', 'CNST',
       'GERN', 'BPMC', 'ALLO', 'BBIO', 'FGEN', 'PGNX', 'MGTX', 'NXTC',
       'IMMU', 'ZIOP', 'CRSP', 'IOVA', 'VKTX', 'EIDX', 'MYOV', 'AMRS',
       'KRYS', 'KURA', 'MDGL', 'UBX', 'TCDA', 'QURE', 'MRTX', 'ASMB',
       'GLYC', 'RYTM', 'FIXX', 'DRNA', 'ARWR', 'ALLK', 'GOSS', 'WVE']

In [ ]:
with open('1_31_2020_pull.json', 'r') as file:
    data = json.load(file)

In [ ]:
start = datetime.datetime.now()
period = [1, 2]
for stock in stocks:
    
    stock_dict_list = data[stock]
    num_days = len(stock_dict_list)
    for index, dict_ in enumerate(stock_dict_list):
        _open = dict_['Open']
        if index+period[1] >= num_days:
            _low = None
            _max = None
        else:
            _low = np.min([dict_['Low'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
            _max = np.max([dict_['High'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
        stock_dict_list[index]['Low_1'] = _low
        stock_dict_list[index]['Max_1'] = _max
        
        #Moving Average 10
        if index- 10 >=0:
            average = np.mean([dict_['Open'] for dict_ in stock_dict_list[index-10:index]])
        else:
            average = None
        stock_dict_list[index]['Moving10'] = average
        
        #Moving Average 30
        if index- 30 >=0:
            average = np.mean([dict_['Open'] for dict_ in stock_dict_list[index-30:index]])
        else:
            average = None
        stock_dict_list[index]['Moving30'] = average
        
    data[stock] = stock_dict_list

# # 3 Day Max/Min Value (% Difference from Closing Price)
# # The range is from the 2nd day to the 3rd after the input day
# start = datetime.datetime.now()
# period = [2, 4]
# for stock in stocks:
    
#     stock_dict_list = data[stock]
#     num_days = len(stock_dict_list)
#     for index, dict_ in enumerate(stock_dict_list):
#         _open = dict_['Open']
#         if index+period[1] >= num_days:
#             _low = None
#             _max = None
#         else:
#             _low = np.min([dict_['Low'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
#             _max = np.max([dict_['High'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
#         stock_dict_list[index]['Low_3'] = _low
#         stock_dict_list[index]['Max_3'] = _max
#     data[stock] = stock_dict_list

# # 10 Day Max/Min Value (% Difference from Closing Price)
# # The range is from the 4th day to the 10th after the input day

# start = datetime.datetime.now()
# period = [4, 11]
# for stock in stocks:
    
#     stock_dict_list = data[stock]
#     num_days = len(stock_dict_list)
#     for index, dict_ in enumerate(stock_dict_list):
#         _open = dict_['Open']
#         if index+period[1] >= num_days:
#             _low = None
#             _max = None
#         else:
#             _low = np.min([dict_['Low'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
#             _max = np.max([dict_['High'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
#         stock_dict_list[index]['Low_10'] = _low
#         stock_dict_list[index]['Max_10'] = _max
#     data[stock] = stock_dict_list

# # 30 Day Max/Min Value (% Difference from Closing Price)
# # The range is from the 11th day to the 30th after the input day
# start = datetime.datetime.now()
# period = [11, 31]
# for stock in stocks:
    
#     stock_dict_list = data[stock]
#     num_days = len(stock_dict_list)
#     for index, dict_ in enumerate(stock_dict_list):
#         _open = dict_['Open']
#         if index+period[1] >= num_days:
#             _low = None
#             _max = None
#         else:
#             _low = np.min([dict_['Low'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
#             _max = np.max([dict_['High'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
#         stock_dict_list[index]['Low_30'] = _low
#         stock_dict_list[index]['Max_30'] = _max
#     data[stock] = stock_dict_list

In [ ]:
for stock in stocks:
    df = pd.DataFrame(data[stock])
    max_value = df[['High', 'Low', 'Open', 'Close', 'Adj Close', 'Moving10', 'Moving30']].max()
    df[['High', 'Low', 'Open', 'Close', 'Adj Close', 'Moving10', 'Moving30']] = df[['High', 'Low', 'Open', 'Close', 'Adj Close', 'Moving10', 'Moving30']]/max_value
    df['Volume'] = df['Volume']/ df['Volume'].max()
    df= df.dropna()
    
    data[stock] = df.to_dict('records')

In [ ]:
data_in_list = []
for stock in stocks:
    stock_dict_list = data[stock]
    for index, dict_ in enumerate(stock_dict_list):
        list_ = [0, 0, 0, 0, 0, 0, 0, 0]
        if dict_['Max_1'] <= 0:
            list_[0] = 1      
        else:
            if dict_['Max_1'] <= 0.02:
                list_[1] = 1
            else:
                if dict_['Max_1'] <= 0.04:
                    list_[2] = 1
                else:
                    if dict_['Max_1'] <= 0.06:
                        list_[3] = 1
                    else:
                        if dict_['Max_1'] <= 0.08:
                            list_[4] = 1
                        else:
                            if dict_['Max_1'] <= 0.10:
                                list_[5] = 1
                            else:
                                if dict_['Max_1'] < 0.12:
                                    list_[6] = 1
                                else:
                                    list_[7] = 1
        stock_dict_list[index]['High_1'] = list_
        dict_['High_1'] = list_
        data_in_list.append(dict_)
    data[stock] = stock_dict_list
df = pd.DataFrame(data_in_list) 
df.describe()

In [ ]:
processed_data = {}

for stock in stocks:
    x = []
    target = []
    dict_ = {}
    for i in data[stock]:
        row =np.array([i['High'], i['Low'], i['Open'], i['Close'], i['Adj Close'], i['Volume'], i['Moving10'], i['Moving30']])
        x.append(row)
        row = np.array(i['High_1'])
        target.append(row)
    dict_['Data'] = x
    dict_['Target'] = target
    processed_data[stock] = dict_

In [ ]:
for stock in stocks:
    length_ = len(processed_data[stock]['Data'])
    length_test = len(processed_data[stock]['Data'])//8
    test_start = length_- 50
    test_end = length_test+ test_start
    processed_data[stock]['data_1'] = np.array(processed_data[stock]['Data'][:test_start])
    processed_data[stock]['data_1'] =   processed_data[stock]['data_1'].reshape(len(processed_data[stock]['data_1']), 1, 8)
    processed_data[stock]['data_test'] = np.array(processed_data[stock]['Data'][test_start:-10])
    processed_data[stock]['data_test'] = processed_data[stock]['data_test'].reshape(len(processed_data[stock]['data_test']),1,8)
    transfer_list = np.array(processed_data[stock]['Target'][:test_start])
    processed_data[stock]['target_1'] =transfer_list.reshape(len(transfer_list), 8)
    transfer_list = np.array(processed_data[stock]['Target'][test_start:-10])
    processed_data[stock]['target_test'] = transfer_list.reshape(len(transfer_list),8)

In [ ]:
model = models.Sequential()
model.add(layers.GRU(50, return_sequences=True, input_shape=(1,8)))
model.add(layers.Dropout(0.2))
model.add(layers.GRU(1000, return_sequences=False))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(8, activation = "softmax"))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 100
for stock in stocks:
    (feature_train, label_train, feature_test, label_test) = (processed_data[stock]['data_1'],processed_data[stock]['target_1'], 
                                                                    processed_data[stock]['data_test'],
                                                              processed_data[stock]['target_test']) 
    model.fit(feature_train, label_train, batch_size=25, epochs=epochs, 
              validation_data = (feature_test, label_test), verbose=0)
    dict_ = model.history.history
    plt.figure(figsize=(8,5))
    plt.title(f"{stock} {epochs} Epochs Loss")
    plt.plot(dict_['loss'], label = 'Loss')
    plt.plot(dict_['val_loss'], label= 'Validation Loss')
    plt.legend(bbox_to_anchor=(1.05, 1))
    plt.show()
    plt.figure(figsize=(8,5))
    plt.title(f"{stock} {epochs} Epochs Accuracy")
    plt.plot(dict_['acc'], label = 'Accuracy')
    plt.plot(dict_['val_acc'], label= 'Validation Accuracy')
    plt.legend(bbox_to_anchor=(1.05, 1))
    plt.show() 

In [ ]:
epochs = 1000

    

stock = 'SGMO'
(feature_train, label_train, feature_test, label_test) = (processed_data[stock]['data_1'],processed_data[stock]['target_1'], 
                                                                processed_data[stock]['data_test'], processed_data[stock]['target_test']) 
model.fit(feature_train, label_train, batch_size=25, epochs=epochs, validation_data = (feature_test, label_test), verbose=0)
dict_ = model.history.history
plt.figure(figsize=(8,5))
plt.title(f"{stock} {epochs} Epochs Loss")
plt.plot(dict_['loss'], label = 'Loss')
plt.plot(dict_['val_loss'], label= 'Validation Loss')
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()
plt.figure(figsize=(8,5))
plt.title(f"{stock} {epochs} Epochs Accuracy")
plt.plot(dict_['acc'], label = 'Accuracy')
plt.plot(dict_['val_acc'], label= 'Validation Accuracy')
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()
filepath = f"Model_GRU_cat_High1{epochs}.h5"
    
model.save_weights(filepath)



In [ ]:
from sklearn.metrics import label_ranking_average_precision_score, coverage_error, label_ranking_average_precision_score

In [ ]:
list_= []
predict_list = []
true_list = []
for stock in stocks:
    X = np.array(processed_data[stock]['Data']).reshape(len(processed_data[stock]['Data']), 1, 8)
    predict = model.predict(X)
    for index, i in enumerate(predict):
        predict_list.append(i)
        true_list.append(processed_data[stock]['Target'][index])
    list_.append({'Ticker':stock, 'Ranking':label_ranking_average_precision_score(processed_data[stock]['Target'], predict)})
                  
df = pd.DataFrame(list_)
df.sort_values('Ranking', ascending=False).head(15)

In [ ]:
columns=['Under 0%', '0.5%', '1.5%', '3%', '5%', '7%', '9%', 'Over 10%']

for column, name in enumerate(columns):
    
    transfer_list = []
    for index, target in enumerate(true_list):
        if target[column] == 1:
            transfer_list.append(predict_list[index])
    plt.figure(figsize=(8,5))
    pd.DataFrame(transfer_list, columns=columns).mean().plot.bar().set_title(name)
    plt.vlines(column, 0,.5)
    plt.show

In [ ]:

X = np.array(processed_data['SGMO']['Data']).reshape(len(processed_data['SGMO']['Data']), 1, 8)
predict = model.predict(X)
transfer_list = []
for column, name in enumerate(columns):
    for index, target in enumerate(processed_data['SGMO']['Target']):
        if target[column] == 1:
            transfer_list.append(predict[index])
    plt.figure(figsize=(8,5))
    pd.DataFrame(transfer_list, columns=columns).mean().plot.bar().set_title(name)
    plt.vlines(column, 0,.5)
    plt.show   
                  
